In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, DateType, DoubleType
from pyspark.sql import functions
import math
import plotly.offline as pyo
import plotly.graph_objs as go
spark = SparkSession \
    .builder \
    .appName("airlinedelay") \
    .config("spark.executor.memory", "2G") \
    .config("spark.executor.cores", "4") \
    .getOrCreate()

context = spark.sparkContext

/home/ubuntu/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning:

Python 3.6 support is deprecated in Spark 3.2.



In [2]:
orgSchema = StructType([
    StructField("FL_DATE", DateType(), True),
    StructField("OP_CARRIER", StringType(), True),
    StructField("OP_CARRIER_FL_NUM", IntegerType(), True),
    StructField("ORIGIN", StringType(), True),
    StructField("DEST", StringType(), True),
    StructField("CRS_DEP_TIME", DoubleType(), True),
    StructField("DEP_TIME", DoubleType(), True),
    StructField("DEP_DELAY", DoubleType(), True),
    StructField("TAXI_OUT", DoubleType(), True),
    StructField("WHEELS_OFF", DoubleType(), True),
    StructField("WHEELS_ON", DoubleType(), True),
    StructField("TAXI_IN", DoubleType(), True),
    StructField("CRS_ARR_TIME", DoubleType(), True),
    StructField("ARR_TIME", DoubleType(), True),
    StructField("ARR_DELAY", DoubleType(), True),
    StructField("CANCELLED", DoubleType(), True),
    StructField("CANCELLATION_CODE", StringType(), True),
    StructField("DIVERTED", DoubleType(), True),
    StructField("CRS_ELAPSED_TIME", DoubleType(), True),
    StructField("ACTUAL_ELAPSED_TIME", DoubleType(), True),
    StructField("AIR_TIME", DoubleType(), True),
    StructField("DISTANCE", DoubleType(), True),
    StructField("CARRIER_DELAY", DoubleType(), True),
    StructField("WEATHER_DELAY", DoubleType(), True),
    StructField("NAS_DELAY", DoubleType(), True),
    StructField("SECURITY_DELAY", DoubleType(), True),
    StructField("LATE_AIRCRAFT_DELAY", DoubleType(), True)
])

newSchema = StructType([
    StructField("FL_DATE", DateType(), True),
    StructField("OP_CARRIER", StringType(), True),
    StructField("OP_CARRIER_FL_NUM", IntegerType(), True),
    StructField("ORIGIN", StringType(), True),
    StructField("DEST", StringType(), True),
    StructField("DEP_TIME", DoubleType(), True),
    StructField("DEP_DELAY", DoubleType(), True),
    StructField("TAXI_OUT", DoubleType(), True),
    StructField("WHEELS_OFF", DoubleType(), True),
    StructField("WHEELS_ON", DoubleType(), True),
    StructField("TAXI_IN", DoubleType(), True),
    StructField("ARR_TIME", DoubleType(), True),
    StructField("ARR_DELAY", DoubleType(), True),
    StructField("ARR_TIME2", DoubleType(), True),
    StructField("DISTANCE", DoubleType(), True),
    StructField("CARRIER_DELAY", DoubleType(), True),
    StructField("WEATHER_DELAY", DoubleType(), True),
    StructField("NAS_DELAY", DoubleType(), True),
    StructField("SECURITY_DELAY", DoubleType(), True),
    StructField("LATE_AIRCRAFT_DELAY", DoubleType(), True)
])

dfOrgSchema = spark.read \
    .option("sep", ",") \
    .option("header", True) \
    .schema(orgSchema) \
    .csv("/home/ubuntu/data/org-schema/*.csv")

dfNewSchema = spark.read \
    .option("sep", ",") \
    .option("header", True) \
    .schema(newSchema) \
    .csv("/home/ubuntu/data/new-schema/*.csv") 

dfCorrupted = spark.read \
    .option("sep", ",") \
    .option("header", False) \
    .schema(newSchema) \
    .csv("/home/ubuntu/data/corrupted/*.csv") 

for column in dfOrgSchema.columns:
    if not column in dfNewSchema.columns:
        dfNewSchema = dfNewSchema.withColumn(column, functions.lit(None))
        
for column in dfNewSchema.columns:
    if not column in dfOrgSchema.columns:
        dfOrgSchema = dfOrgSchema.withColumn(column, functions.lit(None))
        
df = dfOrgSchema.unionByName(dfNewSchema, allowMissingColumns=True).unionByName(dfCorrupted, allowMissingColumns=True)        
df.createOrReplaceTempView("airlinedelay")

In [3]:
df.count()

71526575